# code to retrack data from old btrack versions

First, make sure you have the latest version of btrack installed:

```sh
pip install --upgrade -e git+git://github.com/quantumjot/BayesianTracker.git
```

In [1]:
import os
import napari
import btrack
print(btrack.__version__)
import numpy as np

import csv, glob
from tqdm.notebook import tqdm_notebook
import time

0.4.2


# Define list of experiments from csv file

In [2]:
expt_list = []
with open('/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/control_expts.csv', newline='') as csvfile:
            control_expts_csv = csv.reader(csvfile, delimiter='\n')#, quotechar='|')
            for row in control_expts_csv:
                expt_list.append(row[0])
expt_list

['GV0794/Pos2',
 'GV0794/Pos9',
 'GV0794/Pos14',
 'GV0795/Pos2',
 'GV0795/Pos9',
 'GV0795/Pos15',
 'GV0796/Pos2',
 'GV0796/Pos9',
 'GV0796/Pos17',
 'GV0796/Pos18',
 'GV0797/Pos2',
 'GV0797/Pos9',
 'GV0797/Pos16',
 'GV0797/Pos17',
 'GV0798/Pos2',
 'GV0798/Pos9',
 'GV0798/Pos16',
 'GV0798/Pos17',
 'GV0800/Pos2',
 'GV0800/Pos9',
 'GV0800/Pos14',
 'GV0801/Pos2',
 'GV0802/Pos2',
 'GV0804/Pos2',
 'GV0805/Pos2',
 'GV0806/Pos2',
 'GV0807/Pos2',
 'GV0801/Pos8',
 'GV0804/Pos10',
 'GV0804/Pos11',
 'GV0805/Pos8',
 'GV0806/Pos8',
 'GV0801/Pos7',
 'GV0804/Pos8',
 'GV0804/Pos9',
 'GV0805/Pos7',
 'GV0806/Pos7',
 'GV0801/Pos6',
 'GV0804/Pos6',
 'GV0804/Pos7',
 'GV0805/Pos6',
 'GV0806/Pos6']

# Define filenames of models to use

In [3]:
obj_types = ['obj_type_1', 'obj_type_2']
obj_models = [
    'MDCK_config_new.json',
    'MDCK_config_new.json'
]

# Iterate over all experiments

In [ ]:
root_path = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/'
for expt in tqdm_notebook(expt_list, total = len(expt_list)):
    try:
        expt_ID = expt.split('/')[0]
        pos_ID = expt.split('/')[1]
        PATH = os.path.join(root_path, expt_ID, pos_ID, pos_ID+'_aligned/HDF')
        OLDFILE = glob.glob(os.path.join(PATH, '*.hdf5'))[0]
        NEWFILE = os.path.join(PATH, 'retracked_{}.h5'.format(time.strftime("%Y%m%d-%H%M")))
        if os.path.exists(OLDFILE):
            print(OLDFILE, 'found')
        else:
            print(OLDFILE, 'NOT FOUND')

        print('Loading old tracks', OLDFILE)
        with btrack.dataio.HDF5FileHandler(OLDFILE, 'r') as hdf:
            segmentation = hdf.segmentation
        print('Calculating new tracks')    
        
        for obj_type, obj_model in zip(obj_types, obj_models):

            with btrack.dataio.HDF5FileHandler(OLDFILE, 'r', obj_type=obj_type) as hdf:
                objects = hdf.objects
                area = hdf._hdf['objects'][obj_type]['area'][:]

            for idx, obj in enumerate(objects):
                obj.properties = {'area': area[idx]}

            # make a filtered list of objects for tracking
            filtered_objects = [o for o in objects if o.properties['area']>100.]

            with btrack.BayesianTracker() as tracker:

                # configure the tracker using a config file
                tracker.configure_from_file(
                    os.path.join("/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/Bayesian_config", obj_model)
                )
                tracker.max_search_radius = 50

                # append the objects to be tracked
                tracker.append(filtered_objects)

                # set the volume
                tracker.volume=((0, 1600), (0, 1200), (-1e5, 1e5))

                # track them (in interactive mode)
                tracker.track_interactive(step_size=100)

                # generate hypotheses and run the global optimizer
                tracker.optimize()

                tracker.export(
                    NEWFILE, 
                    obj_type=obj_type, 
                    filter_by='area>100'
                )

                tracks = tracker.tracks
        print('Saving new tracks', NEWFILE)
        with btrack.dataio.HDF5FileHandler(NEWFILE, 'a') as hdf:
            hdf.write_segmentation(segmentation)
            print('btrack version: ', hdf._hdf['tracks']['obj_type_1'].attrs['version'])
    except:
        print(OLDFILE, 'failed')

  0%|          | 0/42 [00:00<?, ?it/s]

[INFO][2021/07/30 07:11:18 PM] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/segmented.hdf5...


/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/segmented.hdf5 found
Loading old tracks /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/segmented.hdf5


[INFO][2021/07/30 07:11:22 PM] Loading segmentation (955, 1200, 1600)
[INFO][2021/07/30 07:11:22 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/segmented.hdf5
[INFO][2021/07/30 07:11:22 PM] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/segmented.hdf5...
[INFO][2021/07/30 07:11:22 PM] Loading objects/obj_type_1 (7173, 5) (7173 filtered: None)
[INFO][2021/07/30 07:11:22 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/segmented.hdf5
[INFO][2021/07/30 07:11:22 PM] Loaded btrack: /home/nathan/anaconda3/lib/python3.7/site-packages/btrack/libs/libtracker.so
[INFO][2021/07/30 07:11:22 PM] btrack (v0.4.2) library imported
[INFO][2021/07/30 07:11:22 PM] Setting max XYZ search radius to: 100
[INFO][2021/07/30 07:11:22 PM

Calculating new tracks


...

[INFO][2021/07/30 07:20:27 PM] Loading segmentation (679, 1200, 1600)
[INFO][2021/07/30 07:20:27 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0797/Pos16/Pos16_aligned/HDF/segmented.hdf5
[INFO][2021/07/30 07:20:27 PM] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0797/Pos16/Pos16_aligned/HDF/segmented.hdf5...
[INFO][2021/07/30 07:20:27 PM] Loading objects/obj_type_1 (504, 5) (504 filtered: None)
[INFO][2021/07/30 07:20:27 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0797/Pos16/Pos16_aligned/HDF/segmented.hdf5
[INFO][2021/07/30 07:20:27 PM] Loaded btrack: /home/nathan/anaconda3/lib/python3.7/site-packages/btrack/libs/libtracker.so
[INFO][2021/07/30 07:20:27 PM] btrack (v0.4.2) library imported
[INFO][2021/07/30 07:20:27 PM] Setting max XYZ search radius to: 100
[INFO][2021/07/30 07:20:2

Calculating new tracks
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0797/Pos16/Pos16_aligned/HDF/segmented.hdf5 failed
/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0797/Pos17/Pos17_aligned/HDF/segmented.hdf5 found
Loading old tracks /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0797/Pos17/Pos17_aligned/HDF/segmented.hdf5


[INFO][2021/07/30 07:20:30 PM] Loading segmentation (676, 1200, 1600)
[INFO][2021/07/30 07:20:30 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0797/Pos17/Pos17_aligned/HDF/segmented.hdf5
[INFO][2021/07/30 07:20:30 PM] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0797/Pos17/Pos17_aligned/HDF/segmented.hdf5...
[INFO][2021/07/30 07:20:30 PM] Loading objects/obj_type_1 (6216, 5) (6216 filtered: None)
[INFO][2021/07/30 07:20:30 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0797/Pos17/Pos17_aligned/HDF/segmented.hdf5
[INFO][2021/07/30 07:20:30 PM] Loaded btrack: /home/nathan/anaconda3/lib/python3.7/site-packages/btrack/libs/libtracker.so
[INFO][2021/07/30 07:20:30 PM] btrack (v0.4.2) library imported
[INFO][2021/07/30 07:20:30 PM] Setting max XYZ search radius to: 100
[INFO][2021/07/30 07:20

Calculating new tracks


[INFO][2021/07/30 07:20:30 PM] Optimization complete. (Solution: optimal)
[INFO][2021/07/30 07:20:30 PM]  - Fates.FALSE_POSITIVE: 1 (of 314)
[INFO][2021/07/30 07:20:31 PM]  - Fates.LINK: 3 (of 174)
[INFO][2021/07/30 07:20:31 PM]  - Fates.DIVIDE: 0 (of 30)
[INFO][2021/07/30 07:20:31 PM]  - Fates.APOPTOSIS: 105 (of 106)
[INFO][2021/07/30 07:20:31 PM]  - Fates.INITIALIZE_BORDER: 310 (of 313)
[INFO][2021/07/30 07:20:31 PM]  - Fates.TERMINATE_BORDER: 205 (of 313)
[INFO][2021/07/30 07:20:31 PM]  - TOTAL: 1250 hypotheses
[INFO][2021/07/30 07:20:31 PM] Completed optimization with 311 tracks
[INFO][2021/07/30 07:20:31 PM] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0797/Pos17/Pos17_aligned/HDF/retracked_20210730-1920.h5...
[INFO][2021/07/30 07:20:31 PM] Writing objects/obj_type_1
[INFO][2021/07/30 07:20:31 PM] Writing labels/obj_type_1
[INFO][2021/07/30 07:20:31 PM] Writing properties/obj_type_1
[INFO][2021/07/30 07:20:31 PM] Wr

# Run on single file

In [ ]:
PATH = "/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0806/Pos6/Pos6_aligned/HDF"
OLDFILE = os.path.join(PATH, 'segmented_old.hdf5')
NEWFILE = os.path.join(PATH, 'retracked_new.h5')

### grab the segmentation

In [7]:
with btrack.dataio.HDF5FileHandler(OLDFILE, 'r') as hdf:
    segmentation = hdf.segmentation

[INFO][2021/07/30 06:45:15 PM] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/segmented.hdf5...
[INFO][2021/07/30 06:45:19 PM] Loading segmentation (955, 1200, 1600)
[INFO][2021/07/30 06:45:19 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/segmented.hdf5


### iterate over the cell types, reorganise the data and retrack, writing out the files

In [13]:
for obj_type, obj_model in zip(obj_types, obj_models):

    with btrack.dataio.HDF5FileHandler(OLDFILE, 'r', obj_type=obj_type) as hdf:
        objects = hdf.objects
        area = hdf._hdf['objects'][obj_type]['area'][:]
        
    for idx, obj in enumerate(objects):
        obj.properties = {'area': area[idx]}
        
    # make a filtered list of objects for tracking
    filtered_objects = [o for o in objects if o.properties['area']>100.]
    
    with btrack.BayesianTracker() as tracker:

        # configure the tracker using a config file
        tracker.configure_from_file(
            os.path.join("/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/Bayesian_config/", obj_model)
        )
        tracker.max_search_radius = 50

        # append the objects to be tracked
        tracker.append(filtered_objects)

        # set the volume
        tracker.volume=((0, 1600), (0, 1200), (-1e5, 1e5))

        # track them (in interactive mode)
        tracker.track_interactive(step_size=100)

        # generate hypotheses and run the global optimizer
        tracker.optimize()

        tracker.export(
            NEWFILE, 
            obj_type=obj_type, 
            filter_by='area>100'
       )
        
        tracks = tracker.tracks

[INFO][2021/07/30 06:48:39 PM] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/segmented.hdf5...
[INFO][2021/07/30 06:48:39 PM] Loading objects/obj_type_1 (7173, 5) (7173 filtered: None)
[INFO][2021/07/30 06:48:39 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/segmented.hdf5
[INFO][2021/07/30 06:48:39 PM] Loaded btrack: /home/nathan/anaconda3/lib/python3.7/site-packages/btrack/libs/libtracker.so
[INFO][2021/07/30 06:48:39 PM] btrack (v0.4.2) library imported
[INFO][2021/07/30 06:48:39 PM] Setting max XYZ search radius to: 100
[INFO][2021/07/30 06:48:39 PM] Starting BayesianTracker session
[INFO][2021/07/30 06:48:39 PM] Loading configuration file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/Bayesian_config/MDCK_config_new.json
[INFO][2021/07/30 06:48:39 PM] L

[INFO][2021/07/30 06:48:42 PM]  - Stats (Active: 72, Lost: 12935, Conflicts resolved: 3363)
[INFO][2021/07/30 06:48:42 PM] Tracking objects in frames 900 to 955 (of 955)...
[INFO][2021/07/30 06:48:42 PM]  - Timing (Bayesian updates: 1.04ms, Linking: 0.25ms)
[INFO][2021/07/30 06:48:42 PM]  - Probabilities (Link: 1.00000, Lost: 1.00000)
[INFO][2021/07/30 06:48:42 PM] SUCCESS.
[INFO][2021/07/30 06:48:42 PM]  - Found 15329 tracks in 955 frames (in 0.0s)
[INFO][2021/07/30 06:48:42 PM]  - Inserted 4394 dummy objects to fill tracking gaps
[INFO][2021/07/30 06:48:42 PM] Loading hypothesis model: MDCK_hypothesis_Kristina
[INFO][2021/07/30 06:48:42 PM] Calculating hypotheses (relax: False)...
[INFO][2021/07/30 06:48:42 PM] Setting up constraints matrix for global optimisation...
[INFO][2021/07/30 06:48:58 PM] Optimizing...
[INFO][2021/07/30 06:48:59 PM] Optimization complete. (Solution: optimal)
[INFO][2021/07/30 06:48:59 PM]  - Fates.FALSE_POSITIVE: 2206 (of 15329)
[INFO][2021/07/30 06:48:59 PM

### finally, write the segmentation back to the file

In [14]:
with btrack.dataio.HDF5FileHandler(NEWFILE, 'a') as hdf:
    hdf.write_segmentation(segmentation)
    print('btrack version: ', hdf._hdf['tracks']['obj_type_1'].attrs['version'])

[INFO][2021/07/30 06:49:05 PM] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/retracked.h5...


OSError: Unable to create file (unable to open file: name = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0794/Pos2/Pos2_aligned/HDF/retracked.h5', errno = 17, error message = 'File exists', flags = 15, o_flags = c2)

In [7]:
objects[0]

,ID,x,y,z,t,dummy,states,label,prob,area
0,0,95.124084,972.700745,0.0,0,False,0,4,0.0,411
